In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Input, Multiply

In [ ]:
def SqueezeAndExcitation(inputs, ratio=8):
    b, _, _, c = inputs.shape
    x = GlobalAveragePooling2D()(inputs)
    x = Dense(c//ratio, activation="relu", use_bias=False)(x)
    x = Dense(c, activation="sigmoid", use_bias=False)(x)
    #x = inputs * x
    x = Multiply()([x, inputs])
    return x


In [ ]:
import os
import pandas as pd

# set path to folder containing image files
folder_path = 'input/utk-face-cropped/utkcropped/'

# create an empty list to store file names and ages
file_list = []

# loop through files in the folder
for filename in os.listdir(folder_path):
    # print(filename)
    if filename.endswith('.jpg'):  
        age = int(filename.split('_')[0])
        file_list.append((filename, age))

        
df = pd.DataFrame(file_list, columns=['filename', 'age'])

# print the dataframe
df.head()


In [ ]:
pd.set_option('display.max_rows', None)
print(df['age'].value_counts().sort_index(axis = 0))

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils, layers, Input, Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout


np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)
print("Training set shape: ", train.shape)
print("Testing set shape: ", test.shape)

In [ ]:

folder_path_facial_age = '/kaggle/input/facial-age/face_age/'


file_list_facial_age = []

# loop through files in the folder
for foldername in os.listdir(folder_path_facial_age):
    # print(foldername)
    for filename_facial_age in os.listdir(folder_path_facial_age+foldername):
        if filename_facial_age.endswith('.png'):  
            age = int(foldername)
            file_list_facial_age.append((filename_facial_age, age))
        
df_facial_age = pd.DataFrame(file_list_facial_age, columns=['filename', 'age'])

# print the dataframe
train = train.append(df_facial_age, ignore_index=True)
train

In [ ]:
import imgaug.augmenters as iaa
from PIL import Image
import numpy as np

save_folder_path = 'working/augmented/'
os.mkdir(save_folder_path)



# create image augmenter object
seq1 = iaa.Sequential([
    iaa.Fliplr(1),  # apply horizontal flipping with 50% probability
])
seq2 = iaa.Sequential([
    iaa.Affine(rotate=(-20)),  # apply 20-degree clockwise or anti-clockwise rotation
])
seq3 = iaa.Sequential([
    iaa.Affine(rotate=(20)),  # apply 20-degree clockwise or anti-clockwise rotation
])
seq4 = iaa.Sequential([
    iaa.Affine(rotate=(-40)),  # apply 20-degree clockwise or anti-clockwise rotation
])
seq5 = iaa.Sequential([
    iaa.Affine(rotate=(20)),  # apply 20-degree clockwise or anti-clockwise rotation
])

# loop through rows in the dataframe
for index, row in train.iterrows():
    # get the file name and label from the dataframe
    filename = row['filename']
    label = row['age']
    # open the image file
    img = Image.open(os.path.join(folder_path, filename))
    # convert the image to numpy array
    img_array = np.array(img)
    Image.fromarray(img_array).save(os.path.join(save_folder_path, "augmented0_" + filename))
    # apply image augmentations
    img_aug2 = seq2(image=img_array)
    # save the augmented image to a file
    Image.fromarray(img_aug2).save(os.path.join(save_folder_path, "augmented1_" + filename))
    
    img_aug3 = seq3(image=img_array)
    Image.fromarray(img_aug3).save(os.path.join(save_folder_path, "augmented2_" + filename))
    
    img_aug4 = seq4(image=img_array)
    Image.fromarray(img_aug4).save(os.path.join(save_folder_path, "augmented3_" + filename))
    
    img_aug5 = seq5(image=img_array)
    Image.fromarray(img_aug5).save(os.path.join(save_folder_path, "augmented4_" + filename))
    
    img_aug6 = seq1(image=img_array)
    Image.fromarray(img_aug6).save(os.path.join(save_folder_path, "augmented5_" + filename))
    
    img_aug7 = seq2(image=img_aug6)
    Image.fromarray(img_aug7).save(os.path.join(save_folder_path, "augmented6_" + filename))
    
    img_aug8 = seq3(image=img_aug6)
    Image.fromarray(img_aug8).save(os.path.join(save_folder_path, "augmented7_" + filename))
    
    img_aug9 = seq4(image=img_aug6)
    Image.fromarray(img_aug9).save(os.path.join(save_folder_path, "augmented8_" + filename))
    
    img_aug10 = seq5(image=img_aug6)
    Image.fromarray(img_aug10).save(os.path.join(save_folder_path, "augmented9_" + filename))


In [ ]:
train_aug_df = pd.DataFrame
test_df = pd.DataFrame

In [ ]:
folder_path = 'working/augmented/'

file_list = []

# loop through files in the folder
for filename in os.listdir(folder_path):
    # print(filename)
    if filename.endswith('.jpg'):  
        age = int(filename.split('_')[1])
    
        file_list.append((filename, age))
        
        
train_df = pd.DataFrame(file_list, columns=['filename', 'age'])

# print the dataframe
train_df.head()

In [ ]:
train_aug_df = train_df.copy()
test_df = test.copy()

In [ ]:
train_aug_df = train.copy()
test_df = test.copy()

In [ ]:
train_df = train.copy()

In [ ]:
print(len(train_aug_df))
print(len(test_df))

In [ ]:
new_folder_name = 'input/utk-face-cropped/utkcropped'
def add_folder_name(row):
    return f"{new_folder_name}/{row['filename']}"

# apply the function to the 'File Name' column in the dataframe
train_aug_df['filename'] = train_df.apply(add_folder_name, axis=1)



new_folder_name = 'input/utk-face-cropped/utkcropped'
def add_ftest_folder_name(row):
    return f"{new_folder_name}/{row['filename']}"

test_df['filename'] = test_df.apply(add_ftest_folder_name, axis=1)

print(train_aug_df.head())
print(test_df.head())

print(len(train_aug_df))
print(len(test_df))

In [ ]:
new_folder_name = '/working/augmented'
def add_folder_name(row):
    return f"{new_folder_name}/{row['filename']}"


train_aug_df['filename'] = train_df.apply(add_folder_name, axis=1)



new_folder_name = '/input/utk-face-cropped/utkcropped'
def add_ftest_folder_name(row):
    return f"{new_folder_name}/{row['filename']}"

test_df['filename'] = test_df.apply(add_ftest_folder_name, axis=1)

print(train_aug_df.head())
print(test_df.head())

print(len(train_aug_df))
print(len(test_df))

In [ ]:

def class_labels_reassign(age):
    if age == 1:
        return 0
    elif age == 2:
        return 1
    elif age == 3:
        return 2
    elif age == 4:
        return 3
    elif age == 5:
        return 4
    elif age == 6:
        return 5
    elif age == 7:
        return 6
    elif age == 8:
        return 7
    elif age == 9:
        return 8
    elif age == 10:
        return 9
    elif age == 11:
        return 10
    elif age == 12:
        return 11
    elif age == 13:
        return 12
    elif age == 14:
        return 13
    elif age == 15:
        return 14
    elif age == 16:
        return 15
    elif age == 17:
        return 16
    elif age == 18:
        return 17
    elif age == 19:
        return 18
    elif age == 20:
        return 19
    elif age == 21:
        return 20
    elif age == 22:
        return 21
    elif age == 23:
        return 22
    elif age == 24:
        return 23
    elif age == 25:
        return 24
    elif age == 26:
        return 25
    elif age == 27:
        return 26
    elif age == 28:
        return 27
    elif age == 29:
        return 28
    elif age == 30:
        return 29
    elif age == 31:
        return 30
    elif age == 32:
        return 31
    elif age == 33:
        return 32
    elif age == 34:
        return 33
    elif age == 35:
        return 34
    elif age == 36:
        return 35
    elif age == 37:
        return 36
    elif age == 38:
        return 37
    elif age == 39:
        return 38
    elif age == 40:
        return 39
    elif age == 41:
        return 40
    elif age == 42:
        return 41
    elif age == 43:
        return 42
    elif age == 44:
        return 43
    elif age == 45:
        return 44
    elif age == 46:
        return 45
    elif age == 47:
        return 46
    elif age == 48:
        return 47
    elif age == 49:
        return 48
    elif age == 50:
        return 49
    elif age == 51:
        return 50
    elif age == 52:
        return 51
    elif age == 53:
        return 52
    elif age == 54:
        return 53
    elif age == 55:
        return 54
    elif age == 56:
        return 55
    elif age == 57:
        return 56
    elif age == 58:
        return 57
    elif age == 59:
        return 58
    elif age == 60:
        return 59
    elif age == 61:
        return 60
    elif age == 62:
        return 61
    elif age == 63:
        return 62
    elif age == 64:
        return 63
    elif age == 65:
        return 64
    elif age == 66:
        return 65
    elif age == 67:
        return 66
    elif age == 68:
        return 67
    elif age == 69:
        return 68
    elif age == 70:
        return 69
    elif age == 71:
        return 70
    elif age == 72:
        return 71
    elif age == 73:
        return 72
    elif age == 74:
        return 73
    elif age == 75:
        return 74
    elif age == 76:
        return 75
    elif age == 77:
        return 76
    elif age == 78:
        return 77
    elif age == 79:
        return 78
    elif age == 80:
        return 79
    elif age == 81:
        return 80
    elif age == 82:
        return 81
    elif age == 83:
        return 82
    elif age == 84:
        return 83
    elif age == 85:
        return 84
    elif age == 86:
        return 85
    elif age == 87:
        return 86
    elif age == 88:
        return 87
    elif age == 89:
        return 88
    elif age == 90:
        return 89
    elif age == 91:
        return 90
    elif age == 92:
        return 91
    elif age == 93:
        return 92
#     elif age == 94:
#         return 93
    elif age == 95:
        return 93
    elif age == 96:
        return 94
#     elif age == 97:
#         return 95
#     elif age == 98:
#         return 97
    elif age == 99:
        return 95
    elif age == 100:
        return 96
    elif age == 101:
        return 97
    elif age == 103:
        return 98
    elif age == 105:
        return 99
    elif age == 110:
        return 100
    elif age == 111:
        return 101
    elif age == 115:
        return 102
    else:
        return 103
        
train_aug_df['target'] = train_aug_df['age'].map(class_labels_reassign)
test_df['target'] = test_df['age'].map(class_labels_reassign)

In [ ]:
pd.set_option('display.max_rows', None)
print(train_aug_df.info())
print(len(train_aug_df['age'].value_counts().sort_index(axis = 0)))
print(train_aug_df['target'].value_counts().sort_index(axis = 0))

In [ ]:
# Converting the filenames and target class labels into lists for augmented train and test datasets.
train_aug_filenames_list = list(train_aug_df['filename'])
train_aug_labels_list = list(train_aug_df['target'])
test_filenames_list = list(test_df['filename'])
test_labels_list = list(test_df['target'])

# Creating tensorflow constants of filenames and labels for augmented train and test datasets from the lists defined above.
train_aug_filenames_tensor = tf.constant(train_aug_filenames_list)
train_aug_labels_tensor = tf.constant(train_aug_labels_list)
test_filenames_tensor = tf.constant(test_filenames_list)
test_labels_tensor = tf.constant(test_labels_list)

In [ ]:

num_classes = 104
def _parse_function(filename, label):   
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)    
    # image_resized = tf.image.resize(image_decoded, [128, 128])
    # label = tf.one_hot(label, num_classes)
    label = label
    return image_decoded, label

train_aug_dataset = tf.data.Dataset.from_tensor_slices((train_aug_filenames_tensor, train_aug_labels_tensor))
train_aug_dataset = train_aug_dataset.map(_parse_function)
# train_aug_dataset = train_aug_dataset.repeat(3)
train_aug_dataset = train_aug_dataset.batch(128)   

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames_tensor, test_labels_tensor))
test_dataset = test_dataset.map(_parse_function)
# test_dataset = test_dataset.repeat(3)
test_dataset = test_dataset.batch(128)    

In [ ]:
train_aug_dataset
image_batch, label_batch = next(iter(train_aug_dataset))
print(image_batch.shape)
print(label_batch)

In [ ]:
test_dataset
image_batch, label_batch = next(iter(test_dataset))
print(image_batch.shape)
print(label_batch)

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Attention, Dropout
from keras.models import Model

# Define the input layer
inputs = layers.Input(shape=(200, 200, 1))

# Define the first set of CNN layers
conv1 = layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs)
avg_pool1 = layers.AveragePooling2D(pool_size=(2,2))(conv1)

conv2 = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(avg_pool1)
avg_pool2 = layers.AveragePooling2D(pool_size=(2,2))(conv2)

conv3 = layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(avg_pool2)
avg_pool3 = layers.AveragePooling2D(pool_size=(2,2))(conv3)

conv4 = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(avg_pool3)
avg_pool4 = layers.AveragePooling2D(pool_size=(2,2))(conv4)



attn1 = SqueezeAndExcitation(avg_pool4)

x = layers.GlobalAveragePooling2D()(attn1)


x = layers.Dense(256, activation='relu')(x)

x = layers.Dense(256, activation='relu')(x)

# Add the final output layer
outputs = layers.Dense(104, activation='softmax')(x)

# Create the model
final_cnn = Model(inputs, outputs, name='cnn_with_attention')

final_cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy',])

final_cnn.summary()


In [ ]:
from tqdm.keras import TqdmCallback
from keras.callbacks import ReduceLROnPlateau


checkpoint_filepath = 'working/best_age_model_attention'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=7,
    verbose=2
)

reduce_learning_rate = ReduceLROnPlateau(
                                    monitor="val_loss",
                                    patience=3, 
                                    episilon= 0.01,
                                    factor=0.1,  
                                    cooldown = 4, 
                                    verbose=2)
                            
# Fitting the above created CNN model.
final_cnn_history = final_cnn.fit(train_aug_dataset,
                                  batch_size=128,
                                  validation_data=test_dataset,
                                  epochs=50,
                                  callbacks=[reduce_learning_rate, model_checkpoint_callback, early_stopping, tensorboard,
                                             TqdmCallback()],
                                  verbose=2,
                                  shuffle=False    
                                 )

In [ ]:

final_cnn.load_weights(checkpoint_filepath)

In [ ]:
# Checking the train and test loss and accuracy values from the neural network above.
train_loss = final_cnn_history.history['loss']
test_loss = final_cnn_history.history['val_loss']
train_accuracy = final_cnn_history.history['sparse_categorical_accuracy']
test_accuracy = final_cnn_history.history['val_sparse_categorical_accuracy'] 

# Plotting a line chart to visualize the loss and accuracy values by epochs.
fig, ax = plt.subplots(ncols=2, figsize=(15,7))
ax = ax.ravel()
ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)
ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Loss', fontsize=14)
ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)
ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)
ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)
ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)
fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of our model by epochs", fontsize=16)


In [ ]:
# Evaluating the model on test dataset.
final_cnn_score = final_cnn.evaluate(test_dataset, verbose=1)
# Printing the relevant score summary.
final_cnn_labels = final_cnn.metrics_names
print(f'CNN model {final_cnn_labels[0]} \t\t= {round(final_cnn_score[0], 3)}')
print(f'CNN model {final_cnn_labels[1]} \t= {round(final_cnn_score[1], 3)}')


# Generating predictions from the model above.
final_cnn_pred = final_cnn.predict(test_dataset)
final_cnn_pred = final_cnn_pred.argmax(axis=-1)

In [ ]:
chk = pd.DataFrame()
chk['prediction'] = final_cnn_pred;
chk['actual'] = test_labels_list;
chk.head(50)

In [ ]:
chk = pd.DataFrame()
chk['prediction'] = final_cnn_pred;
chk['actual'] = test_labels_list;
chk.head(50)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
mae(test_labels_list, final_cnn_pred)